In [9]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import pandas as pd
import time
import os
import warnings
warnings.filterwarnings("ignore")

In [117]:
#url 내에서 상세 정보 크롤링 해보기
options = webdriver.ChromeOptions()
#options.headless = True
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
browser = webdriver.Chrome(options=options)
browser.maximize_window()

txt = pd.read_csv('LCK_2015_Summer_Playoffs_match_links.txt', header=None)
url_list = [i for i in txt.iloc[:,0]]
url = url_list[0]

browser.get(url)

In [107]:
#text를 통해 얻을 수 있는 정보들(level, player_name, kda, damage, dpm, dtpm, sight, cs)
text_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').get_attribute('innerText').split('\n\n')
text_data = [i.strip().split('\n\t\n') for i in text_data[1:len(text_data)//2+1]] + [i.strip().split('\n\t\n') for i in text_data[len(text_data)//2+1:]]

data = []
for i in range(len(text_data)):
        level = text_data[i][0]
        player_name = text_data[i][1]
        k,d,a = text_data[i][2].split('\n')[0].split(' / ')
        kda_score = text_data[i][2].split('\n')[1]
        damage = text_data[i][3]
        dpm = text_data[i][4]
        dtpm = text_data[i][5]
        sight = text_data[i][6].split(sep='\n')[0]
        ward_install = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[0]
        ward_erase = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[1]
        cs = text_data[i][7].split(sep='\n')[0]
        cs_per_minute = text_data[i][7].split(sep='\n')[1]
        
        data.append([player_name, level, k, d, a, kda_score, damage, dpm, dtpm, 
                     sight, ward_install, ward_erase, cs, cs_per_minute])
        
pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,SKT MaRin,17,4,2,7,5.5,"13,403",425,586,1,21,2,269,8.5
1,SKT Bengi,15,4,1,6,10,"8,896",282,451,4,25,12,100,3.2
2,SKT Faker,17,3,1,5,8,"18,899",599,371,5,7,9,338,10.7
3,SKT Bang,15,4,2,9,6.5,"17,448",553,308,2,5,12,233,7.4
4,SKT Wolf,14,0,0,12,14.4,"1,864",59,369,8,37,9,35,1.1
5,KT Ssumday,15,0,2,2,1,"8,277",262,541,2,26,4,221,7.0
6,KT Score,12,0,4,2,0.5,"3,629",115,622,3,23,6,90,2.9
7,KT Nagne,15,3,4,0,0.75,"10,298",326,483,4,19,6,229,7.3
8,KT Arrow,14,3,2,1,2,"16,937",537,456,2,6,3,265,8.4
9,KT Piccaboo,12,0,3,5,1.66,"5,608",178,499,5,48,11,38,1.2


In [114]:
#받은 데미지
xpath = "//span[contains(text(), '받은 데미지')]"
damage_button = browser.find_element(By.XPATH, xpath)
damage_button.click()

damage_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').find_elements(By.TAG_NAME, 'li')
damage_received = [i.get_attribute('innerText').split('\n')[-1] for i in damage_data]
damage_received

['18.5k',
 '14.2k',
 '11.7k',
 '9.7k',
 '11.6k',
 '17.1k',
 '19.6k',
 '15.2k',
 '14.4k',
 '15.7k']

In [123]:
#tag를 통해 얻을 수 있는 정보(champion, summoner_skill, rune, items)
data = []

tag_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').find_elements(By.TAG_NAME, 'tbody')
tag_data_list = [i.find_elements(By.TAG_NAME, 'tr') for i in tag_data]
tag_data = tag_data_list[0] + tag_data_list[1]
for tag in tag_data:
    champion_data = tag.find_elements(By.TAG_NAME, 'td')[0]
    champion = str(champion_data.find_element(By.TAG_NAME, 'img').get_attribute('src')).split('.')[-2].split('/')[-1]
    summoner_skill = sorted([tag.find_elements(By.TAG_NAME, 'img')[1].get_attribute('title'), 
            str(tag.find_elements(By.TAG_NAME, 'img')[2].get_attribute('title'))]) 
    rune = sorted([tag.find_element(By.XPATH, './td[1]/div/div[1]/div[2]').get_attribute('alt')])
    
    items = tag.find_elements(By.TAG_NAME, 'td')[-1]
    items = items.find_elements(By.TAG_NAME, 'img')
    items = [item.get_attribute('alt') for item in items]
    
    data.append([champion, summoner_skill, rune, items])
    
pd.DataFrame(data)

,0,1,2,3
0,Maokai,"[순간이동, 점멸]",[None],"[도란의 반지, 얼어붙은 심장]"
1,Elise,"[강타, 점멸]",[None],"[쇠사슬 조끼, 마법무효화의 망토, 헤르메스의 발걸음]"
2,Azir,"[방어막, 점멸]",[None],"[도란의 반지, 쓸데없이 큰 지팡이, 라일라이의 수정홀, 내셔의 이빨, 마법사의 신발]"
3,Ashe,"[점멸, 회복]",[None],"[롱소드, 도란의 검, 광전사의 군화, 최후의 속삭임]"
4,Alistar,"[점멸, 탈진]",[None],"[얼음 방패, 예언자의 렌즈]"
5,Gnar,"[순간이동, 점멸]",[None],"[거인의 허리띠, 도란의 검, 파수꾼의 갑옷, 헤르메스의 발걸음, 밴시의 장막]"
6,Vi,"[강타, 점멸]",[None],"[루비 수정, 장화, 밴시의 장막, 예언자의 렌즈]"
7,Leblanc,"[점멸, 점화]",[None],"[방출의 마법봉, 증폭의 고서, 라바돈의 죽음모자, 마법사의 신발]"
8,Corki,"[점멸, 회복]",[None],"[곡괭이, 도란의 검, 마법사의 신발, 삼위일체, 피바라기]"
9,Braum,"[점멸, 점화]",[None],"[기동력의 장화, 강철의 솔라리 펜던트, 예언자의 렌즈]"


In [124]:
browser.quit()

In [165]:
#현재 디렉토리 내 txt 파일 모두 불러오기(리그별)
def get_txt_files(path, league):
    txt_file_list = [file for file in os.listdir(path) if (file.endswith('txt') and file.startswith(f'{league}'))]
    return txt_file_list

def get_urls(txt):
    try:
        url_list = [i for i in pd.read_csv(txt, header=None).iloc[:,0]]
        return url_list
    except:
        return [None]
    
LCK_txt, LPL_txt, LCS_txt, LEC_txt = [], [], [], []
LCK_txt += get_txt_files('./', 'LCK')
print(f'LCK txt 개수 : {len(LCK_txt)}')

LCK_url, LPL_url, LCS_url, LEC_url = [], [], [], []
for txt in LCK_txt:
    urls = get_urls(txt)
    LCK_url += urls
print(f'LCK url 개수 : {len(LCK_url)}')

LCK txt 개수 : 48
LCK url 개수 : 1471


In [166]:
from tqdm import tqdm
final_data = []

for url in tqdm(LCK_url):
        options = webdriver.ChromeOptions()
        options.headless = True
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36')
        browser = webdriver.Chrome(options=options)
        browser.maximize_window()

        browser.get(url)

        data = []

        xpath = "//span[contains(text(), '받은 데미지')]"
        damage_button = browser.find_element(By.XPATH, xpath)
        damage_button.click()

        damage_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').find_elements(By.TAG_NAME, 'li')
        damage_received = [i.get_attribute('innerText').split('\n')[-1] for i in damage_data]

        #사이드창이 있다면 닫기
        xpath = "//span[contains(text(), '닫기')]"
        close_buttons = browser.find_elements(By.XPATH, xpath)
        if len(close_buttons) != 0:
                for close_button in close_buttons:
                        close_button.click()
        browser.get(url)
        time.sleep(1)

        text_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').get_attribute('innerText').split('\n\n')
        text_data = [i.strip().split('\n\t\n') for i in text_data[1:len(text_data)//2+1]] + [i.strip().split('\n\t\n') for i in text_data[len(text_data)//2+1:]]

        tag_data = browser.find_element(By.XPATH, '//*[@id="matchHistoryWrapper"]/div[2]/div[2]').find_elements(By.TAG_NAME, 'tbody')
        tag_data_list = [i.find_elements(By.TAG_NAME, 'tr') for i in tag_data]
        tag_data = tag_data_list[0] + tag_data_list[1]

        for i, tag in enumerate(tag_data):
                level = text_data[i][0]
                player_name = text_data[i][1]
                k,d,a = text_data[i][2].split('\n')[0].split(' / ')
                kda_score = text_data[i][2].split('\n')[1]
                damage_inflicted = text_data[i][3]
                dpm = text_data[i][4]
                dtpm = text_data[i][5]
                sight = text_data[i][6].split(sep='\n')[0]
                ward_install = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[0]
                ward_erase = text_data[i][6].split(sep='\n')[1].split(sep=' / ')[1]
                cs = text_data[i][7].split(sep='\n')[0]
                cs_per_minute = text_data[i][7].split(sep='\n')[1]
                
                champion_data = tag.find_elements(By.TAG_NAME, 'td')[0]
                champion = str(champion_data.find_element(By.TAG_NAME, 'img').get_attribute('src')).split('.')[-2].split('/')[-1]
                summoner_skill = sorted([tag.find_elements(By.TAG_NAME, 'img')[1].get_attribute('title'), 
                        str(tag.find_elements(By.TAG_NAME, 'img')[2].get_attribute('title'))]) 
                rune = sorted([tag.find_element(By.XPATH, './td[1]/div/div[1]/div[2]').get_attribute('alt')])
                
                items = tag.find_elements(By.TAG_NAME, 'td')[-1]
                items = items.find_elements(By.TAG_NAME, 'img')
                items = [item.get_attribute('alt') for item in items]
                
                final_data.append([player_name, champion, level, summoner_skill, rune,
                                k, d, a, kda_score, damage_inflicted, damage_received[i], dpm, dtpm, sight, ward_install, ward_erase,
                                cs, cs_per_minute, items])
                
pd.DataFrame(data)

  0%|          | 0/1471 [00:22<?, ?it/s]


IndexError: list index out of range